In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
import seaborn as sns

import math
from math import sqrt

from numpy import absolute
from numpy import mean
from numpy import std

from sklearn import metrics
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn import neighbors


import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError 
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.random import set_seed

import xgboost as xgb


#### There is a lot of data, which will require a lot of computing resources, so I took only a part of it

In [77]:
#pickup_datetime was represented as an object so I converted it to a time variable
train = pd.read_csv("../input/new-york-city-taxi-fare-prediction/train.csv", nrows = 100000, parse_dates=["pickup_datetime"]
                   )
test = pd.read_csv("../input/new-york-city-taxi-fare-prediction/test.csv")

In [78]:
print(train.shape)
print(test.shape)

In [79]:
train.head()

In [80]:
train.dtypes

In [81]:
train.describe()

## From this we learn that
* The minimum fare is negative, which is impossible
* Some travel points are missing the city
* The maximum number of passengers is equal to 208, which is impossible
* The maximum fare also unreal

# Data Cleaning & Feature Engineering

#### Find, if we have data that has no value

In [82]:
print(train.isnull().sum())

In [83]:
train = train.dropna(how = 'any', axis = 'rows')

In [84]:
print('Old size: %d' % len(train))

#### There will be no negative tax and you may not be able to pay more than a certain limit depending on the circumstances, let's say this limit is 200\\$ 
Also, the minimum fare for a New York taxi is 2,50\\$  

In [85]:
train = train.drop(train[train.fare_amount<2.5].index, axis = 0)
train = train.drop(train[train.fare_amount>300].index, axis = 0)

#### Delete the data whose passenger_count exceeded 6, because it cannot physically fit more in the taxi and it is not allowed. Taxi can also move without passenger and carry cargo, so lets permit passenger_count == 0 data


In [86]:
train = train.drop(train[train['passenger_count']>6].index, axis = 0)
train = train.drop(train[train['passenger_count']<0].index, axis = 0)

#### Remove the pickup_latitude data whose values are greater than 90 and less than -90, because latitudes are between -90 and 90 degrees

In [87]:
train = train.drop(train[train['pickup_latitude']<-90].index, axis = 0)
train = train.drop(train[train['pickup_latitude']>90].index, axis = 0)

#### Remove the pickup_longitude data whose values are greater than 180 and less than -180, because longitudes are between -180 and 180 degrees

In [88]:
train = train.drop(train[train['pickup_longitude']<-180].index, axis = 0)
train = train.drop(train[train['pickup_longitude']>180].index, axis = 0)

#### Repeat the same for dropoff_latitude and dropoff_longitude

In [89]:
train = train.drop(train[train['dropoff_latitude']<-90].index, axis = 0)
train = train.drop(train[train['dropoff_latitude']>90].index, axis = 0)

train = train.drop(train[train['dropoff_longitude']<-180].index, axis = 0)
train = train.drop(train[train['dropoff_longitude']>180].index, axis = 0)

#### The geographical location may correspond to the real one, but not to New York area, so let's filter the existing data. For this, let's introduce the conditional city limits

In [90]:
def select_outside_boundingbox(df, BB):
    filter_df = df.loc[(df['pickup_longitude'] < BB[0]) | (df['pickup_longitude'] > BB[1]) | \
           (df['pickup_latitude'] < BB[2]) | (df['pickup_latitude'] > BB[3]) | \
           (df['dropoff_longitude'] < BB[0]) | (df['dropoff_longitude'] > BB[1]) | \
           (df['dropoff_latitude'] < BB[2]) | (df['dropoff_latitude'] > BB[3])]
    
    return filter_df

NYC_BB = (-74.5, -72.8, 40.5, 41.8)

In [91]:
outliers = select_outside_boundingbox(train, NYC_BB)
outliers

In [92]:
train = train.drop(outliers.index, axis = 0)

In [93]:
print('New size: %d' % len(train))

In [94]:
#key column and pickup_datetime column of Test set should also be presented as time data
#test['key'] = pd.to_datetime(test['key'])
#test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])

In [95]:
test.dtypes

#### We can understand displacement through start and end points.

#### We will use the Haversine formula to calculate the distance between two geolocations

In [96]:
#საწყისი და საბოლოო კოორდინატების წარმოვადგინე tuple სახით
train["loc1"] = train[["pickup_latitude","pickup_longitude"]].apply(tuple, axis=1)
train["loc2"] = train[["dropoff_latitude","dropoff_longitude"]].apply(tuple, axis=1)

In [97]:
import haversine as hs

        
train['H_Distance'] = train.apply(lambda row: hs.haversine(row.loc1,row.loc2), axis=1)

#### Let's also calculate the distance using the Chebyshev method

In [98]:
def chebyshev(pickup_long, dropoff_long, pickup_lat, dropoff_lat):
    return np.maximum(np.absolute(pickup_long - dropoff_long), np.absolute(pickup_lat - dropoff_lat))

train['Chebyshev'] = chebyshev(train['pickup_longitude'], train['dropoff_longitude'], train['pickup_latitude'], train['dropoff_latitude'])

In [99]:
train.head()

In [100]:
train["hour"] = train.pickup_datetime.dt.hour
train["day_of_week"] = train.pickup_datetime.dt.weekday
train["day_of_month"] = train.pickup_datetime.dt.day
train["week"] = train.pickup_datetime.dt.week
train["month"] = train.pickup_datetime.dt.month
train["year"] = train.pickup_datetime.dt.year - 2000

train['minute'] =train['pickup_datetime'].dt.minute
train['second'] = train['pickup_datetime'].dt.second
train['dayofyear'] = train['pickup_datetime'].dt.dayofyear

In [101]:
train.head()

#### Add a variable that determines how much each kilometer of travel costs

In [102]:
train["fare_to_dist_ratio"] = train["fare_amount"] / ( train["H_Distance"]+0.0001)

#### Remove those whose start and end points match

In [103]:
train = train.drop(train[train['loc1']==train['loc2']].index, axis = 0)


#### It is necessary to take into account the circumstances of calling at the airport, according to which the fee changes dramatically. For this, let's write the given function that shows how many kilometers away the point is from the airport


In [104]:
def add_distances_from_airport(dataset):
    #coordinates of all these airports
    jfk_coords = (40.639722, -73.778889)
    ewr_coords = (40.6925, -74.168611)
    lga_coords = (40.77725, -73.872611)

    dataset['pickup_jfk_distance'] = dataset.apply(lambda row: hs.haversine(jfk_coords,row.loc1), axis=1)
    dataset['dropof_jfk_distance'] = dataset.apply(lambda row: hs.haversine(jfk_coords,row.loc2), axis=1)
    
    
    dataset['pickup_ewr_distance'] = dataset.apply(lambda row: hs.haversine(ewr_coords,row.loc1), axis=1)
    dataset['dropof_ewr_distance'] = dataset.apply(lambda row: hs.haversine(ewr_coords,row.loc2), axis=1)
    
    
    dataset['pickup_lga_distance'] = dataset.apply(lambda row: hs.haversine(lga_coords,row.loc1), axis=1)
    dataset['dropof_lga_distance'] = dataset.apply(lambda row: hs.haversine(lga_coords,row.loc2), axis=1)

    return dataset


train = add_distances_from_airport(train)


### Let's repeat the same procedures on the test part

In [105]:
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])

test["loc1"] = test[["pickup_latitude","pickup_longitude"]].apply(tuple, axis=1)
test["loc2"] = test[["dropoff_latitude","dropoff_longitude"]].apply(tuple, axis=1)

test['H_Distance'] = test.apply(lambda row: hs.haversine(row.loc1,row.loc2), axis=1)

test['Chebyshev'] = chebyshev(test['pickup_longitude'], test['dropoff_longitude'], test['pickup_latitude'], test['dropoff_latitude'])


test["hour"] = test.pickup_datetime.dt.hour
test["day_of_week"] = test.pickup_datetime.dt.weekday
test["day_of_month"] = test.pickup_datetime.dt.day
test["week"] = test.pickup_datetime.dt.week
test["month"] = test.pickup_datetime.dt.month
test["year"] = test.pickup_datetime.dt.year - 2000

test['minute'] =test['pickup_datetime'].dt.minute
test['second'] = test['pickup_datetime'].dt.second
test['dayofyear'] = test['pickup_datetime'].dt.dayofyear

test = add_distances_from_airport(test)

In [106]:
#დიდ მონაცემთა ბაზას ჭირდება დიდი გამოთვლითი რესურსი, მოცემული ფუნქციით, რიცხვით ტიპებს დავიყვანთ შესაძლო მინიმალურ მოცულობამდე,რაც გაამარტივებს გამოთვლებს
def downcast(df):

    df_int = df.select_dtypes(include=['int64', 'int32', 'int16', 'int8', 'int'])
    df[df_int.columns] = df_int.apply(pd.to_numeric,downcast='unsigned')
    
    df_float = df.select_dtypes(include=['float64', 'float32', 'float16', 'float'])
    df[df_float.columns] = df_float.apply(pd.to_numeric,downcast='float')
        
    return df
downcast(train)
downcast(test)
train.dtypes

## Map visualization


In [107]:
train.plot(y='pickup_latitude',x='pickup_longitude',kind="scatter",alpha=0.7,s=0.02)
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
plt.title("Pickups Data")

plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

In [108]:
train.plot(y='dropoff_latitude',x='dropoff_longitude',kind="scatter",alpha=0.5,s=0.02)
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
plt.title("Dropoff Data")

plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

In [109]:
import folium


long_trips=train[train['H_Distance']>=10]

drop_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12)

### For each pickup point add a circlemarker

for index, row in long_trips.iterrows():
    
    folium.CircleMarker([row['dropoff_latitude'], row['dropoff_longitude']],
                        radius=3,
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(drop_map)
for index, row in long_trips.iterrows():
    
    folium.CircleMarker([row['pickup_latitude'], row['pickup_longitude']],
                        radius=3,
                        color="blue", 
                        fill_opacity=0.9
                       ).add_to(drop_map)
drop_map

## EDA

In [110]:
sns.histplot(data=train,x="fare_amount",kde=True)

plt.axvline(train["fare_amount"].mean(),color = "k",
            linestyle = "dashed",label = "Avg trip fare ($)")
plt.axvline(train["fare_amount"].median(),color = "r",
            linestyle = "dashed",label = "Median trip fare ($)")

plt.title("Distribution of fare amount")
plt.xticks(np.arange(0, 100, step=5))
plt.legend(loc = "best",prop = {"size" : 12})
plt.gcf().set_size_inches(15,8)
plt.xlim(0,100)

##### A right-skewed distribution
* #### Most taxi fares range from $2.5\\$-20\\$.
* #### The average taxi fee varies between 10\\$-12\\$
* #### 45\\$-50\\$-57\\$ peaks are observed, which fixed fee

In [111]:
sns.histplot(data=train,x="H_Distance",kde=True)

plt.axvline(train["H_Distance"].mean(),color = "k",
            linestyle = "dashed",label = "Avg H_Distance (km)")
plt.axvline(train["H_Distance"].median(),color = "r",
            linestyle = "dashed",label = "Median H_Distance (km)")

plt.title("Distribution of fare amount")
plt.xticks(np.arange(0, 30, step=5))
plt.legend(loc = "best",prop = {"size" : 12})
plt.gcf().set_size_inches(15,8)
plt.xlim(0,30)

* #### Passengers travel an average of 3-5 km by taxi

#### 1. Does the number of passengers affect the fare?

In [112]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.histplot(data=train, x="passenger_count", stat="count", discrete=True)
plt.title("Distribution of passenger count")
plt.xlabel('No. of Passengers')
plt.ylabel('Frequency')

* #### Most of the passengers were traveling alone

In [113]:
plt.figure(figsize=(15,8))
plt.scatter(x=train['passenger_count'], y=train['fare_amount'], s=1.5)
plt.xlabel('No. of Passengers')
plt.ylabel('Fare')

* #### One-passenger taxi has more passengers whose fare is higher

#### 2. Does the pick-up date and time affect the fare?

In [114]:
colormap = plt.cm.RdBu
plt.figure(figsize=(20, 20))
heatmap = sns.heatmap(train.corr(),linewidths=0.1,vmax=1.0,vmin=0, 
            square=True, cmap="Blues", linecolor='white', annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [115]:
plt.figure(figsize=(15,8))
plt.scatter(x=train['year'], y=train['fare_amount'], s=1.5)
plt.xlabel('Years')
plt.ylabel('Fare')

In [116]:
train[['fare_amount','year']].corr()

* #### The rate does not change significantly over the years

In [117]:
plt.figure(figsize=(15,8))
plt.scatter(x=train['month'], y=train['fare_amount'], s=1.5)
plt.xlabel('Month')
plt.ylabel('Fare')

* #### The rate is uniform throughout the months

In [118]:
plt.figure(figsize=(15,8))
plt.scatter(x=train['day_of_month'], y=train['fare_amount'], s=1.5)
plt.xlabel('Days')
plt.ylabel('Fare')

* #### The fare is uniform throughout the month

In [119]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.histplot(data=train, x="hour", stat="count", discrete=True, kde=True)
plt.title("Distribution by Hours")
plt.xlabel('Hours')
plt.ylabel('Frequency')

* #### Taxi fares are rare at 5am and reaches the maximum at 7pm

In [120]:
plt.figure(figsize=(15,7))
plt.scatter(x=train['hour'], y=train['fare_amount'], s=1.5)
plt.xlabel('Hour')
plt.ylabel('Fare')

In [121]:
train[train.passenger_count <7][['fare_amount','passenger_count']].corr()


In [122]:
def time_slicer(df, timeframes, value, color="purple"):
    """
    Function to count observation occurrence through different lenses of time.
    """
    f, ax = plt.subplots(len(timeframes), figsize = [12,12])
    for i,x in enumerate(timeframes):
        df.loc[:,[x,value]].groupby([x]).mean().plot(ax=ax[i],color=color)
        ax[i].set_ylabel(value.replace("_", " ").title())
        ax[i].set_title("{} by {}".format(value.replace("_", " ").title(), x.replace("_", " ").title()))
        ax[i].set_xlabel("")
    plt.tight_layout(pad=0)

In [123]:
time_slicer(df=train, timeframes=['hour', 'day_of_week','day_of_month', 'week', 'month', 'year',], value = "fare_amount", color="blue")

* #### The higher the demand, the lower the fee and vice versa
* #### Average fares peak on Mondays and Thursdays
* #### The average fee has been increasing over the years 

In [124]:
time_slicer(df=train, timeframes=['hour', 'day_of_week','day_of_month', 'week', 'month', 'year',], value = "H_Distance", color="green")

In [125]:
train[['fare_amount','H_Distance']].corr()

* #### The correlation number between these two values is high, as a result the graphs are very similar to each other
* #### At 5 o'clock in the morning, some people who have a long distance to travel leave home early, but they are not many, so the fare increases.

In [126]:
time_slicer(df=train, timeframes=['hour', 'day_of_week','day_of_month', 'week', 'month', 'year',], value = "fare_to_dist_ratio", color="red")

* #### The price of 1 km is high at the beginning of the day and decreases during the day
* #### The price of 1 km increases during the week
* #### The price of 1 km decreases during the months

In [127]:
# scatter plot distance - fare
fig, axs = plt.subplots(1, 2, figsize=(16,6))
axs[0].scatter(train.H_Distance, train.fare_amount, alpha=0.2)
axs[0].set_xlabel('H_Distance mile')
axs[0].set_ylabel('fare $USD')
axs[0].set_title('All data')

# zoom in on part of data
idx = (train.H_Distance < 30) & (train.fare_amount < 100)
axs[1].scatter(train[idx].H_Distance, train[idx].fare_amount, alpha=0.2)
axs[1].set_xlabel('H_Distance mile')
axs[1].set_ylabel('fare $USD')
axs[1].set_title('Zoom in on distance < 30 mile, fare < $100');

In [128]:
print("Average $USD/Km : {:0.2f}".format(train.fare_amount.sum()/train.H_Distance.sum()))

* #### The horizontal lines on the graph to the right may indicate fixed fares from the airport
* #### Overall, a linear relationship is observed

In [129]:
train.reset_index(drop=True, inplace=True)

train[train["fare_to_dist_ratio"]>500]

In [130]:
passenger_fare = train.groupby(['passenger_count']).mean()
sns.barplot(x=passenger_fare.index, y=passenger_fare['fare_amount'], palette = "Set3")
plt.xlabel('Number of Passengers')
plt.ylabel('Average Fare Price')
plt.title('Average Fare Price for Number of Passengers')
plt.show()

# Data Preprocess

In [131]:
train.drop(['key', 'pickup_datetime','loc1','loc2',"fare_to_dist_ratio"], axis=1, inplace=True)

test_key = pd.DataFrame(test['key'])
test.drop(['key', 'pickup_datetime','loc1','loc2',], axis=1, inplace=True)


In [132]:
print(train.dtypes)
print(test.dtypes)

#### Split Data

In [133]:
X = train.iloc[:,train.columns != "fare_amount"]
y = train["fare_amount"]
X_test = test
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.15, random_state=42, shuffle=True)


# scaler = MinMaxScaler(feature_range=(0, 1))

# X_train_scaled = scaler.fit_transform(X_train)
# X_train = pd.DataFrame(X_train_scaled)

# X_valid_scaled = scaler.fit_transform(X_valid)
# X_valid = pd.DataFrame(X_valid_scaled)

# X_test_scaled = scaler.fit_transform(X_test)
# X_test = pd.DataFrame(X_test_scaled)

## Model 1 : Linear Regression

In [134]:
# def Linear_reg(X_train, X_valid, y_train, y_valid):    
#     linear = LinearRegression()
#     linear.fit(X_train, y_train)
    
#     y_train_predict = linear.predict(X_train)
#     r2_train = r2_score(y_train, y_train_predict)
#     RMSE_train = mean_squared_error(y_train, y_train_predict, squared=False)
    
#     y_valid_predict = linear.predict(X_valid)
#     r2_valid = r2_score(y_valid, y_valid_predict)
#     RMSE_valid = mean_squared_error(y_valid, y_valid_predict, squared=False)
    
#     return r2_train, r2_valid, linear,RMSE_train,RMSE_valid

# r2_train, r2_valid, linear,RMSE_train,RMSE_valid = Linear_reg(X_train, X_valid, y_train, y_valid)

# #print("R^2 (train) : ", r2_train)
# print("RMSE (train): ", RMSE_train)
# #print("R^2 (valid) : ", r2_valid)
# print("RMSE (valid): ", RMSE_valid)


* RMSE (train):  1.3202917e-05
* RMSE (valid):  1.4740558

## Model 2 : Polynomial Regression


In [135]:
# def Poly_reg(X_train, X_valid, y_train, y_valid, degree={'degree' : 2}):
#     poly = PolynomialFeatures(degree=degree['degree'])
#     X_train_ = poly.fit_transform(X_train)
#     est_poly = LinearRegression()    
#     est_poly.fit(X_train_,y_train) 

#     y_train_predict = est_poly.predict(X_train_)
#     r2_train = r2_score(y_train, y_train_predict)
#     RMSE_train = mean_squared_error(y_train, y_train_predict, squared=False)
    
    
#     X_valid_ = poly.fit_transform(X_valid)

#     y_valid_predict = est_poly.predict(X_valid_)
#     r2_valid = r2_score(y_valid, y_valid_predict)
#     RMSE_valid = mean_squared_error(y_valid, y_valid_predict, squared=False)

#     return r2_train, r2_valid, RMSE_train, RMSE_valid

# for deg in range(1,5):
#     degree={'degree' : deg}
#     r2_train, r2_valid,RMSE_train, RMSE_valid = Poly_reg(X_train, X_valid, y_train, y_valid, degree)
#     print("Polynomial degree", deg)
#     print("R^2 (train) : ", r2_train)
#     print("RMSE (train): ", RMSE_train)
#     print("R^2 (valid) : ", r2_valid)
#     print("RMSE (valid): ", RMSE_valid)

* Polynomial degree 1
* R^2 (train) :  0.9999999999953564
* RMSE (train):  2.0643134e-05
* R^2 (valid) :  0.3986446655631378
* RMSE (valid):  7.342692
* Polynomial degree 2
* R^2 (train) :  0.9999999998963758
* RMSE (train):  9.751603e-05
* R^2 (valid) :  0.40283202490610714
* RMSE (valid):  7.317083
* Polynomial degree 3
* R^2 (train) :  0.999999991415696
* RMSE (train):  0.00088756054
* R^2 (valid) :  0.39955006653268765
* RMSE (valid):  7.3371625

## Model 2 : Ridge Regression

In [136]:

# model = Ridge(alpha=1.0)
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
# scores = absolute(scores)
# print('rmse: %.3f (%.3f)' % (mean(scores), std(scores)))

rmse: 0.041 (0.003)

## Model 3: DECISION TREE REGRESSOR MODEL

In [137]:
# dtr = DecisionTreeRegressor().fit(X_train, y_train)
# y_valid_pred = dtr.predict(X_valid)



# # Root Mean Square Error
# rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
# print("RMSE: %f" % (rmse))

# # Mean Squared Error
# print("Mean squared error: %.2f"
#       % mean_squared_error(y_valid, y_valid_pred))

# # R2 Score
# print('Variance score: %.2f' % r2_score(y_valid, y_valid_pred))

* RMSE: 1.474739
* Mean squared error: 2.17
* Variance score: 0.98
* when size = 1000000

## Model 4: KNR

In [138]:
# rmse_val = [] 
# for K in range(20):
#     K = K+1
#     model = neighbors.KNeighborsRegressor(n_neighbors = K)

#     model.fit(X_train, y_train)  
#     pred=model.predict(X_valid) 
#     error = sqrt(mean_squared_error(y_valid,pred)) 
#     rmse_val.append(error) 
#     print('RMSE value for k= ' , K , 'is:', error)

In [139]:
# curve = pd.DataFrame(rmse_val) 
# curve.plot()

## Model 5: Support Vector Machines

In [140]:
# svr = svm.LinearSVR()
# svr.fit(X_train,y_train)
# test_val_pred_svr = svr.predict(X_test_val)
# np.sqrt(metrics.mean_squared_error(y_test_val,test_val_pred_svr))

## Model 6: Neural Network

In [141]:
# early_stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=1e-3, patience=3)
# callback=[early_stop]
# adam = Adam(lr=0.0001)

# model = Sequential() 
# model.add(Dense(100, activation='relu', input_shape=(train.shape[1],)))
# model.add(Dropout(0.6))
# model.add(Dense(80, activation='relu'))
# model.add(Dropout(0.6))
# model.add(Dense(40, activation='relu'))
# model.add(Dropout(0.6))
# model.add(Dense(1, activation='linear'))
# model.compile(loss='mse', optimizer=adam, metrics=['mae'])
# history = model.fit(X_train,y_train,batch_size=256, epochs=50, verbose=1, callbacks=callback,
#          validation_data=(X_valid, y_valid), shuffle=True)

In [142]:
# # summarize history for loss using learning curve
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [143]:
# y_pred = np.array(model.predict(X_valid))
# print(sqrt(mean_squared_error(y_valid, y_pred)))

## Model 7: XGBoost

In [144]:
params = {
   
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0,
    'verbosity' : 0,
    'random_state' : 42
}

In [145]:
def XGBmodel(X_train,X_valid,y_train,y_valid,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_valid = xgb.DMatrix(X_valid,label=y_valid)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_valid,'valid')])
    return model

model = XGBmodel(X_train,X_valid,y_train,y_valid,params)

In [146]:
y = model.predict(xgb.DMatrix(X_test), ntree_limit = model.best_ntree_limit).tolist()

## Submition

### Model 6: Neural Network

In [147]:
subm =  pd.read_csv('../input/new-york-city-taxi-fare-prediction/sample_submission.csv')
subm.fare_amount = y
subm.to_csv('submission.csv',index=False)
print("Submitted")

In [148]:
fscores = pd.DataFrame({'X': list(model.get_fscore().keys()), 'Y': list(model.get_fscore().values())})
fscores.sort_values(by='Y').plot.bar(x='X')